In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random 
import numpy as np 
from PIL import Image
import os
from scipy.spatial import distance

size_x = 1280
size_y = 720
target_path =  'hd.jpg'
components = [
    'kersti/0.jpg',
    'kersti/1.jpg',
    'kersti/2.jpg'
]
def open_image():
    print("open")
#     imgplot = plt.imshow(img)

    
def draw_one_image_random(result):
    x,y,w,h = random.randrange(size_x),random.randrange(size_y), random.randrange(10,300), random.randrange(10,300)
    img = Image.open(random.choice(components))
    if (x+w < size_x) and (y+h < size_y):
        resized = img.resize((w,h), Image.ANTIALIAS)
        result.paste(resized, (x, y, x+w, y+h))

def mutate( init_pic = None):
    if init_pic == None:
        result = Image.new("RGB", (size_x, size_y))
    else:
        result = init_pic.copy()
    for i in range(100):
        draw_one_image_random(result)
    return result



def mutate_rgb(init_pic = None):
    result = init_pic.copy()
    
    target_pixels = Image.open(target_path).load()
    comp_pixels = result.load()
    
    for i in range(1000000):
        
        select_x = random.randrange(size_x)
        select_y = random.randrange(size_y)
        
        target_rgb = target_pixels[select_x, select_y]
        comp_rgb = comp_pixels[select_x, select_y]
        
        r = (target_rgb[0] - comp_rgb[0]) // 2 + comp_rgb[0]
        g = (target_rgb[1] - comp_rgb[1]) // 2 + comp_rgb[1]
        b = (target_rgb[2] - comp_rgb[2]) // 2 + comp_rgb[2]
        
        comp_pixels[select_x, select_y] = (r, g, b)
    return result


def get_rand_slice_fitness(target, comp):
    select_x = random.randrange(size_x - 20)
    select_y = random.randrange(size_y - 20)

    loc = (select_x, select_y, select_x + 20, select_y + 20)

    comp_slice = comp.crop(loc)
    target_slice = target.crop(loc)

    return fitness2(comp_slice, target_slice), loc, comp_slice


def mutate_swap(init_pic = None):
    result = init_pic.copy()
    target_img = Image.open(target_path)

    for i in range(10):
        
        best_fitness, best_loc, best_slice = get_rand_slice_fitness(target_img, result)
        
        for i in range(3):
            current_fitness, current_loc, current_slice = get_rand_slice_fitness(target_img, result)
            if current_fitness < best_fitness:
                result.paste(current_slice, best_loc)
                result.paste(best_slice, current_loc)
        
    return result


def save_to_file(result, path):
    result.save(os.path.expanduser(path))
    pil_im = Image.open(path)
    display(pil_im)

# result = mutate()
# display(result)

# result = mutate(result)
# display(result)

# result = mutate(result)
# display(result)

# save_to_file(result, "result.jpg")


In [2]:
import imagehash
#lesser better, because it is distance
def fitness(imga,imgb):
#     hash0 = imagehash.average_hash(Image.open('hd.jpg')) 
#     hash1 = imagehash.average_hash(Image.open('result.jpg')) 
    hash0 = imagehash.average_hash(imga) 
    hash1 = imagehash.average_hash(imgb) 
    return hash0-hash1


In [3]:
from numba import jit

@jit
def fitness2_helper(a,b):
    total = 0
    for i in range(size_y):
        for j in range(size_x):
            dst = 0
            for k in range(3):
                dst += (a[i][j][k] - b[i][j][k])**2
            total += (dst)**(1/2)
    return total

def fitness2(a,b):
    array_a = np.asarray(a)
    array_b = np.asarray(b)
    return fitness2_helper(array_a,array_b)


target_img = Image.open(target_path)
result_img = mutate()
# ss = np.asarray(target_img)
# print(ss[0][0])
# dst = distance.euclidean(ss[0][0], ss[100][100])
# print(dst)

%time print(fitness2(result_img, target_img))


225675912.7036294
Wall time: 276 ms


In [4]:
target_img = Image.open(target_path)

def init_first():
    #get first 100 child
    childs = []  
    for i in range(100):
        tmp = mutate()
        print(".",end="")
        childs.append(tmp)
    return childs

def get_fitness_score(generation):
    fitness_scores = []
    for i in generation:
        fitness_score = fitness2(i, target_img)
        fitness_scores.append(fitness_score)
    return fitness_scores
        
def run():
    generation = init_first()

    
def run_iterate(generation):
    fitness_scores = get_fitness_score(generation)
    sorted_index = np.argsort(fitness_scores)
    #choose best 20 from the generation
    bestX = sorted_index[:21]
    #save best one
    
    #every child generate 50 child
    new_generation = []
    
    #insert best 10 of itself
    for i in range(20):
        fitness_score = fitness(generation[bestX[i]], target_img)
        new_generation.append(generation[bestX[i]])
    
    for i in range(20):
        #mutate 5 time
        for j in range(5):
            print(".",end="")
#             new_generation.append(mutate(generation[bestX[i]]))
            new_generation.append(mutate_swap(generation[bestX[i]]))
    
    return new_generation
    #repeat
    
# run()

In [5]:
generation = init_first()
fitness_scores = get_fitness_score(generation)
sorted_index = np.argsort(fitness_scores)
print("best fitness : ",fitness_scores[sorted_index[0]])
# display(generation[sorted_index[0]])

....................................................................................................best fitness :  195510320.3874247


In [6]:
# display(target_img)
# display(generation[sorted_index[0]])
for i in range(10):
    print("iterate : ",i)
    %time generation = run_iterate(generation)
    fitness_scores = get_fitness_score(generation)
    sorted_index = np.argsort(fitness_scores)
    print("best fitness : ",fitness_scores[sorted_index[0]])
#     display(generation[sorted_index[0]])

iterate :  0
....................................................................................................Wall time: 7.4 s
best fitness :  195510320.3874247
iterate :  1
....................................................................................................Wall time: 7.46 s
best fitness :  195358264.05812305
iterate :  2
....................................................................................................Wall time: 7.53 s
best fitness :  195358264.05812305
iterate :  3
....................................................................................................Wall time: 7.46 s
best fitness :  195358264.05812305
iterate :  4
....................................................................................................Wall time: 7.48 s
best fitness :  195332529.26839277
iterate :  5
....................................................................................................Wall time: 7.49 s
best fitness :  195078228.91340822
iterat

In [ ]:
for i in range(20):
    print("iterate : ",10+i)
    generation = run_iterate(generation)
    fitness_scores = get_fitness_score(generation)
    sorted_index = np.argsort(fitness_scores)
    print("best fitness : ",fitness_scores[sorted_index[0]])
    display(generation[sorted_index[0]])

In [ ]:
for i in range(150):
    print("iterate : ",30+i)
    generation = run_iterate(generation)
    fitness_scores = get_fitness_score(generation)
    sorted_index = np.argsort(fitness_scores)
    print("best fitness : ",fitness_scores[sorted_index[0]])
    display(generation[sorted_index[0]])

In [ ]:
save_to_file(generation[sorted_index[0]], "result-150-gen.jpg")